In [1]:
#!/usr/bin/python3
import os
import csv
#import mysql.connector as mc 
import pymysql as mc
import time
import webbrowser
import sys
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import getpass
'''
Here is how this file is structured
1) Take in a connection, a yr and a month and query the DB for
 gameID, current inning, abNum, pitchID, event, runners on base, outs,
 current runs for each team, end of game runs for each team, and game date

2) Read all this stuff into a Pandas dataframe.

3) Go through all possible game/inning combinations, and for each game create a smaller
dataframe, and then for each inning take an even smaller dataframe from that
and pass it into the Inning Class.

4) The sole purpose of the inning class is to get the score at the end of the inning so that
I can use it in the at-bats to find the runs created during an inning.

5) Go through the inning dataframe and for each row, find the last pitch of the at-bat.
This will be the "Action Pitch" which I want to create a class from. Note that the data
contains certain issues such as ejections which result in my data classifying certain rows
as having 3 outs prior to the AB, so I will discard those.

6) Choose the action pitch and create an AB class from it. The AB class will first calculate
which of the 24 states we are in (as defined by Tom Tango in The Book). Then it will calculate
the runs generated from the current AB to the end of the inning. (This is all in generate_ab_list)

7) Call calculate_run_expectancies which generates the average runs generated in each of the
24 situations.

8) Call add_run_expectancies which updates the start and end run expectancies 
for every bat in self.ab_list. This will allow us to generate a "value add above replacement" 
for each batter.

9) We're done! We've now calculated the value above replacement for each at-bat in the sample.
Now we can calculate the weights for each type of hit and perform analysis
'''

'\nHere is how this file is structured\n1) Take in a connection, a yr and a month and query the DB for\n gameID, current inning, abNum, pitchID, event, runners on base, outs,\n current runs for each team, end of game runs for each team, and game date\n\n2) Read all this stuff into a Pandas dataframe.\n\n3) Go through all possible game/inning combinations, and for each game create a smaller\ndataframe, and then for each inning take an even smaller dataframe from that\nand pass it into the Inning Class.\n\n4) The sole purpose of the inning class is to get the score at the end of the inning so that\nI can use it in the at-bats to find the runs created during an inning.\n\n5) Go through the inning dataframe and for each row, find the last pitch of the at-bat.\nThis will be the "Action Pitch" which I want to create a class from. Note that the data\ncontains certain issues such as ejections which result in my data classifying certain rows\nas having 3 outs prior to the AB, so I will discard 

In [2]:
class REMatrix:
    def __init__(self,cnx,startTime,yr=None,mo=None):
        query1=''
        query2=''
        if yr and mo:   #this takes 0.35 seconds 
            query1 = """SELECT Pitch2.gameID AS gameID ,curr_inn FROM Pitch2 
                        INNER JOIN Game ON Pitch2.gameID = Game.gameID
                        WHERE YEAR(gameDate)='{}' AND MONTH(gameDate)='{}'
                        GROUP BY gameID,curr_inn""".format(yr,mo)
                        #this takes 0.30 seconds
            query2 = """SELECT Game.gameID AS gameID,abNum,pitchID,the_event,
                        firstBaseRunner,secondBaseRunner, thirdBaseRunner,outs,
                        home_team_runs,away_team_runs,curr_inn, homeTeamScore,
                        awayTeamScore,gameDate
                        FROM Pitch2 INNER JOIN Game ON Pitch2.gameID = Game.gameID
                        WHERE YEAR(gameDate)='{}' AND MONTH(gameDate)='{}'""".format(yr,mo)
        elif yr:        #this takes 0.91 seconds
            query1 = """SELECT Pitch2.gameID AS gameID ,curr_inn 
                        FROM Pitch2 INNER JOIN Game ON Pitch2.gameID = Game.gameID 
                        WHERE YEAR(gameDate)='{}'
                        GROUP BY gameID,curr_inn""".format(yr)
                        #this takes 1.36 seconds
            query2 = """SELECT Game.gameID AS gameID,abNum,pitchID,
                        the_event,firstBaseRunner,secondBaseRunner, 
                        thirdBaseRunner,outs,home_team_runs, 
                        away_team_runs,curr_inn, homeTeamScore,
                        awayTeamScore,gameDate 
                        FROM Pitch2 INNER JOIN Game 
                        ON Pitch2.gameID = Game.gameID 
                        WHERE YEAR(gameDate)='{}'""".format(yr)
        else:           #this took 7.10 seconds
            query1 = """SELECT Pitch2.gameID AS gameID ,curr_inn
                        FROM Pitch2
                        GROUP BY gameID,curr_inn"""
                        #this took 10.45 seconds
            query2 = """SELECT Pitch2.gameID AS gameID,abNum,pitchID,
                        the_event,firstBaseRunner,secondBaseRunner, 
                        thirdBaseRunner,outs,home_team_runs, away_team_runs,
                        curr_inn, homeTeamScore,awayTeamScore,gameDate 
                        FROM Pitch2 INNER JOIN Game on Pitch2.gameID = Game.gameID"""
        
        self.game_inn_df = pd.read_sql_query(query1,cnx)
        self.df = pd.read_sql_query(query2,cnx)
        self.all_combos = [(item[1],item[2]) for item in self.game_inn_df.itertuples()] 
        self.inning_list = []
        self.ab_list = []
        self.expect_dict = {}
        self.generate_ab_list()
        self.calculate_run_expectancies() #generate 24 expectancies for 24 situations
        self.add_run_expectancies()  
    
    #takes the given run expectancies for each of the 24 situations and assigns a value to each at-bat
    #based off this value
    def add_run_expectancies(self):
        #this loop adds the start run expectancies and next at-bats
        for i,ab in enumerate(self.ab_list):
            start_sit = ab.situation 
            ab.start_run_expectancy = self.expect_dict[start_sit]
            try:
                #if ab's are consecutive and in the same inning, mark a next AB
                if (ab.abNum == self.ab_list[i+1].abNum-1 and
                        ab.inning.inning_number == self.ab_list[i+1].inning.inning_number):
                    ab.next_ab = self.ab_list[i+1]
                else:
                    ab.next_ab = None
            except IndexError as e: #special case for the last item
                ab.next_ab = None
        
        #this adds the end run expectancies
        for i, ab in enumerate(self.ab_list):
            if ab.next_ab and ab.abNum == ab.next_ab.abNum-1:
                #this will generally work except for when a player gets picked off
                #to end the inning (or caught stealing to end inning)
                ab.end_run_expectancy = ab.next_ab.start_run_expectancy
            else:
                ab.end_run_expectancy = 0
        
        #finally calculate the value each batter added per AB
        for ab in self.ab_list:
            ab.calculate_adjusted_runs_created() 

    #create a dataframe of each gameID from self.all_combos, and then within those small dataframes
    #look up the individual innings to get those stats. 
    def generate_ab_list(self):
        previous_game_id = None
        curr_df = None
        for j,item in enumerate(self.all_combos):

            curr_time = time.time()
            if item[0] != previous_game_id:
                previous_game_id = item[0]
                curr_df = self.df[self.df['gameID']==item[0]]
                #print ('this combo took {} seconds'.format(time.time()-curr_time))

            #I separated inn_df and curr_df because of performance issues
            #allows me to look up current inning on a dataframe of just one game (~300Pitches)
            #as opposed to looking at all 5.5 million pitches 
            inn_df = curr_df[curr_df['curr_inn']==item[1]]
            inning_object = Inning(curr_df,item[1],item[0])
            
            self.inning_list.append(inning_object)
            
            for i in range(len(inn_df)):
                try:
                    #if ab number is not the same as the next one, it's last pitch of AB
                    if inn_df.iloc[i,1]<inn_df.iloc[i+1,1]:    
                        try:
                            ab_object = AB(inn_df.iloc[i,:],inning_object)
                            self.ab_list.append(ab_object)
                        #i.e. this is the condition where there's sometimes more than 3 outs...
                        except ValueError as e:
                            print ('there were more than 3 outs or less than 0 outs')
                            continue

                #index error happens on last pitch of inning so this is an "action pitch"
                except IndexError as e:
                    try:
                        ab_object = AB(inn_df.iloc[i,:],inning_object)
                        self.ab_list.append(ab_object)
                    except ValueError as e:
                        print (str(item[0])+' had an AB with less than 0 or more than 3 outs')
                        continue
            

    def calculate_run_expectancies(self):
        situations = [item.situation for item in self.ab_list]
        runs_created = [item.runs_created for item in self.ab_list]
        run_expectancy_df = pd.DataFrame({'situation':situations,'runs_created':runs_created})
        for i in range(24):
            self.expect_dict[i] = round(run_expectancy_df[run_expectancy_df.situation==i].mean()['runs_created'],3)

class Inning:
    def __init__(self,df,inn,gameID):
        self.inning_number = inn
        self.gameID = gameID
        self.home_or_away = self.get_home_or_away()
        self.endScore = self.get_end_score(df)

    def get_home_or_away(self):
        if self.inning_number*2%2 == 1:
            return 'Home'
        else:
            return 'Away'

    def get_end_score(self,df):
        #just look at the last row of the dataframe
        inning_df = df[df['curr_inn']==self.inning_number]
        if self.home_or_away == 'Home':
            return inning_df['home_team_runs'].tail(1).iloc[0]
        else:
            return inning_df['away_team_runs'].tail(1).iloc[0]


class AB:
    #passing in a series here. 
    def __init__(self,df,inning):
        
        self.gameID = df['gameID']
        self.abNum = df['abNum']
        self.outs = df['outs']
        self.runners = (df['firstBaseRunner'],df['secondBaseRunner'],df['thirdBaseRunner'])
        self.event = df['the_event']
        self.inning = inning
        self.starting_runs = None
        self.situation = self.calculate_state()
        self.runs_created  = self.calculate_runs_created(df)
        self.start_run_expectancy = 0
        self.end_run_expectancy = 0
        self.next_ab = None
        self.adjusted_runs_created = None
    
    #THIS PART NEEDS TO BE FIXED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    def calculate_adjusted_runs_created(self):
        if not self.next_ab:
            self.adjusted_runs_created = self.end_run_expectancy - self.start_run_expectancy
        else:
            self.adjusted_runs_created = (self.end_run_expectancy - self.start_run_expectancy + 
                                          self.next_ab.starting_runs -self.starting_runs)
            
    def calculate_runs_created(self,df):
        if self.inning.home_or_away == 'Home':
            self.starting_runs = df['home_team_runs']
            return self.inning.endScore - df['home_team_runs']
        else:
            self.starting_runs = df['away_team_runs']
            return self.inning.endScore - df['away_team_runs']

    #determines which of the 24 runner on base/outs circumstance
    def calculate_state(self):
        total = 0
        if self.runners[0]:
            total+=1
        if self.runners[1]:
            total+=2
        if self.runners[2]:
            total+=4

        if self.outs == 1:
            total+=8
        elif self.outs ==2:
            total+=16
        elif self.outs>=3 or self.outs<0:
            raise ValueError("cant start AB with less than 0 or > 3 outs")
        return total

In [3]:
    cnx = mc.connect(user='akashgoyal',password=os.environ['DB_PASSWORD'],host='stromberg.cs.uchicago.edu',db='mlb_practicum',port=3306)
    startTime= time.time()
    tester2 = REMatrix(cnx,startTime,yr=2017)
    print (time.time()-startTime)

28.573760986328125


In [4]:
first_ab = tester2.ab_list[0]
print ('started with {} outs and the event was a {}. Situation was {}, he created {} runs.'.format(first_ab.outs,
                                                    first_ab.event,first_ab.situation,first_ab.runs_created))
print ('starting RE was {}, end RE was {}. On an adjusted basis he produced {} runs, there were runners {} on base'.format(
first_ab.start_run_expectancy, first_ab.end_run_expectancy,first_ab.adjusted_runs_created,first_ab.runners))

tester2.expect_dict

started with 0 outs and the event was a Flyout. Situation was 0, he created 0 runs.
starting RE was 0.539, end RE was 0.284. On an adjusted basis he produced -0.25500000000000006 runs, there were runners (None, None, None) on base


{0: 0.53900000000000003,
 1: 0.91800000000000004,
 2: 1.161,
 3: 1.46,
 4: 1.4490000000000001,
 5: 1.6359999999999999,
 6: 1.79,
 7: 2.113,
 8: 0.28399999999999997,
 9: 0.53600000000000003,
 10: 0.67100000000000004,
 11: 0.90000000000000002,
 12: 1.0760000000000001,
 13: 1.0649999999999999,
 14: 1.4430000000000001,
 15: 1.5489999999999999,
 16: 0.13,
 17: 0.254,
 18: 0.36499999999999999,
 19: 0.50900000000000001,
 20: 0.38200000000000001,
 21: 0.503,
 22: 0.55000000000000004,
 23: 0.76400000000000001}

In [45]:
D = {}
for ab in tester2.ab_list:
    if not D.get(ab.event):
        D[ab.event] = [1,ab.adjusted_runs_created]
    else:
        D[ab.event][0]+=1
        D[ab.event][1]+=ab.adjusted_runs_created

l1 = []
for ev in D:
    l1.append((D[ev][0],D[ev][1]/D[ev][0],ev))

l1 = sorted(l1,key=lambda x: x[1],reverse=True)
l1 = [(x[2],x[0],"%.2f"%round(x[1],2)) for x in l1]
print (*l1,sep='\n')

('Home Run', 1101, '1.37')
('Triple', 118, '1.04')
('Double', 1475, '0.75')
('Fielders Choice', 19, '0.60')
('Fan interference', 5, '0.53')
('Field Error', 252, '0.48')
('Single', 4634, '0.44')
('Catcher Interference', 8, '0.35')
('Hit By Pitch', 287, '0.33')
('Walk', 2408, '0.31')
('Intent Walk', 169, '0.17')
('Sac Fly', 205, '0.05')
('Sac Bunt', 157, '-0.13')
('Bunt Groundout', 45, '-0.21')
('Groundout', 5672, '-0.23')
('Flyout', 3180, '-0.27')
('Lineout', 1880, '-0.27')
('Strikeout', 6716, '-0.27')
('Batter Interference', 7, '-0.28')
('Pop Out', 1442, '-0.28')
('Runner Out', 67, '-0.30')
('Bunt Lineout', 2, '-0.32')
('Forceout', 591, '-0.33')
('Bunt Pop Out', 24, '-0.44')
('Strikeout - DP', 28, '-0.67')
('Fielders Choice Out', 48, '-0.68')
('Grounded Into DP', 633, '-0.80')
('Double Play', 69, '-0.88')
('Sac Fly DP', 2, '-1.31')
('Triple Play', 1, '-1.46')


In [35]:
for ab in tester2.ab_list:
    if ab.event == "Fielders Choice":
        if ab.next_ab:
            print (get_url(ab.gameID),ab.abNum)
            #print ('for fielders choice: ',ab.outs,ab.next_ab.outs)
    elif ab.event == "Fielders Choice Out":
        pass
        #if ab.next_ab:
            #print ('for fielders choice out: ',ab.outs,ab.next_ab.outs)

http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_29/gid_2017_06_29_lanmlb_anamlb_1/inning/inning_all.xml 27
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_10/gid_2017_06_10_milmlb_arimlb_1/inning/inning_all.xml 37
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_17/gid_2017_06_17_miamlb_atlmlb_1/inning/inning_all.xml 90
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_22/gid_2017_06_22_sfnmlb_atlmlb_1/inning/inning_all.xml 87
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_03/gid_2017_06_03_bosmlb_balmlb_1/inning/inning_all.xml 54
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_04/gid_2017_06_04_bosmlb_balmlb_1/inning/inning_all.xml 12
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_16/gid_2017_06_16_tbamlb_detmlb_1/inning/inning_all.xml 17
http://gd2.mlb.com/components/game/mlb/year_2017/month_06/day_13/gid_2017_06_13_texmlb_houmlb_1/inning/inning_all.xml 34
http://gd2.mlb.com/components/ga

In [24]:
#use this to get the pitch f/x url for a given Retrosheet gameID to find issues
def get_url(gameID):
    cnx1 = mc.connect(user='root',password=os.environ['DB_PASSWORD'],
    host='localhost',database='mlb')
    cursor = cnx1.cursor()
    cursor.execute("SELECT gameID, homeTeam,awayTeam,YEAR(gameDate), MONTH(gameDate),DAY(gameDate),isDoubleHeader FROM Game WHERE gameID='{}'".format(gameID))
    
    theURL=''
    for row in cursor:
        theYear = str(row[3])
        theMonth= 0
        theDate=0
        if row[4]<10:
            theMonth ='0'+str(row[4])
        else:
            theMonth=str(row[4])
        if row[5]<10:
            theDate='0'+str(row[5])
        else:
            theDate=str(row[5])

        gameNum= '1'
        if str(row[6])==2:
            gameNum='2'
        homeTeam=row[1].lower()+'mlb'
        visitTeam=row[2].lower()+'mlb'
        new_path = 'year_'+theYear+'/month_'+theMonth + '/day_'+theDate+'/gid_'+theYear+'_'+theMonth +'_'+theDate+'_'
        new_path = new_path+visitTeam+'_'+homeTeam+'_'+gameNum+'/inning/inning_all.xml'
        theURL='http://gd2.mlb.com/components/game/mlb/'+new_path
    cnx1.close()
    return theURL